In [1]:
import datajoint as dj
import numpy as np
import time

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
ta3 = dj.create_virtual_module('ta3', 'microns_ta3')

Connecting celiib@10.28.0.34:3306


In [3]:
ta3p100.ProofreadLabel()
#ta3p100.CoarseLabel()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices label data for the vertices,triangles label data for the faces,edges label data for the edges,status the index of the status descriptor that can be references by the StatusKey
2,648518346341366885,0.35,computer_Auto,bcelii,2019-01-24 20:35:07,=BLOB=,=BLOB=,=BLOB=,complete
2,648518346341371119,0.35,computer_Auto,bcelii,2019-01-26 20:48:03,=BLOB=,=BLOB=,=BLOB=,partial
2,648518346341371119,0.35,computer_Auto,zhiweid,2019-01-24 01:36:06,=BLOB=,=BLOB=,=BLOB=,complete
2,648518346341388820,0.35,computer_Auto,zhiweid,2019-01-26 14:59:55,=BLOB=,=BLOB=,=BLOB=,complete
2,648518346341393609,0.35,computer_Auto,bcelii,2019-01-24 19:36:28,=BLOB=,=BLOB=,=BLOB=,complete
2,648518346342797556,0.35,computer_Auto,bcelii,2019-01-24 18:49:45,=BLOB=,=BLOB=,=BLOB=,complete
2,648518346342806978,0.35,computer_Auto,bcelii,2019-01-24 18:44:51,=BLOB=,=BLOB=,=BLOB=,complete
2,648518346349386137,0.35,computer_Auto,bcelii,2019-01-24 12:25:57,=BLOB=,=BLOB=,=BLOB=,complete
2,648518346349468264,0.35,computer_Auto,spapadop,2019-01-25 14:20:05,=BLOB=,=BLOB=,=BLOB=,complete
2,648518346349470171,0.35,computer_Auto,spapadop,2019-01-25 14:22:26,=BLOB=,=BLOB=,=BLOB=,complete


In [ ]:
ta3p100.ProofreadLabel() & "segment_id=648518346349472601"

In [4]:
ta3p100.SynapseCompartmentLabel()

segmentation segmentation id,synapse_id synapse index within the segmentation,postsynaptic_label numeric label of the compartment
2,16650,1
2,69735,1
2,83755,1
2,89345,1
2,93929,1
2,144997,1
2,170880,1
2,201756,1
2,228560,1
2,230713,1


In [10]:
ta3p100.SynapseCompartmentLabel() + ta3p100.SynapseSpineLabel()

DataJointError: Union requires the same attributes in both arguments

In [ ]:
ta3p100.SynapseSpineLabel()

In [ ]:
#create the new SPINE cgal table
@schema
class ComponentAutoSegment(dj.Computed):
    definition = """
    # creates the labels for the mesh table
    -> ta3p100.Compartment.Component
    clusters     : tinyint unsigned  #what the clustering parameter was set to
    smoothness   : decimal(3,2)             #what the smoothness parameter was set to, number betwee 0 and 1
    ---
    n_triangles  : int unsigned # number of faces
    seg_group    : longblob     # group segmentation ID's for faces from automatic CGAL segmentation
    sdf          : longblob     #  width values for faces from from automatic CGAL segmentation
    median_sdf   : decimal(6,5) # the median width value for the sdf values
    mean_sdf     : decimal(6,5) #the mean width value for the sdf values
    third_q      : decimal(6,5) #the upper quartile for the mean width values
    ninety_perc  : decimal(6,5) #the 90th percentile for the mean width values
    time_updated : timestamp    # the time at which the segmentation was performed
   
    
   """

In [ ]:
ta3.LabelKey()

In [ ]:
ta3p100.Compartment.Component() 

In [ ]:
ta3p100.Compartment.Component() & "n_triangle_indices>5000" & "compartment_type='Axon-Soma'"

In [ ]:
ta3p100.CoarseLabel() & "segment_id=648518346349471565"

In [ ]:
@schema
class ComponentLabel(dj.Computed):
    definition = """
    # creates the labels for the mesh table
    -> ta3p100.ComponentAutoSegment
    time_updated      :timestamp    # the time at which the component labels were updated
    ---
    n_vertices        :int unsigned #number of vertices in component
    n_triangles       :int unsigned #number of faces in component
    labeled_vertices  :longblob     #indicate which vertices are spine,spine_head,spine_neck otherwise 0
    labeled_triangles :longblob     #indicate which faces are spine,spine_head,spine_neck otherwise 0
    n_heads           :int unsigned #totals the number of heads after classification, helps for optimization
    used_version      :tinyint      #whether this component is used in the final labels or not, 0 no, 1 yes
    
   """

In [ ]:
ta3p100.ComponentLabel()

In [ ]:
#find that all of the neurons were processed
comp_ids = cmp.fetch("segment_id")
#print(len(np.unique(comp_ids)))
comp_unique_ids = np.unique(comp_ids)

In [ ]:
#seem to be missing 3 neurons, find out which ones those are
proof_id = (ta3p100.ProofreadLabel() & "status='complete'" & ta3p100.CoarseLabel.proj()).fetch("segment_id")
missing_ids = set(proof_id).difference(set(comp_unique_ids))

In [ ]:
missing_ids

In [ ]:
missing_ids

In [ ]:
#FIND OUT WHO THE AUTHROS WERE FOR THE MISSING IDS
key = dict(segment_id=648518346349506952,status="complete")
(ta3p100.ProofreadLabel() & key)

In [ ]:
class ComponentAutoSegment(dj.Computed):
    definition = """
    # creates the labels for the mesh table
    -> ta3p100.Compartment.Component
    clusters     : tinyint unsigned  #what the clustering parameter was set to
    smoothness   : decimal(3,2)             #what the smoothness parameter was set to, number betwee 0 and 1
    ---
    n_triangles  : int unsigned # number of faces
    seg_group    : longblob     # group segmentation ID's for faces from automatic CGAL segmentation
    sdf          : longblob     #  width values for faces from from automatic CGAL segmentation
    median_sdf   : decimal(6,5) # the median width value for the sdf values
    mean_sdf     : decimal(6,5) #the mean width value for the sdf values
    third_q      : decimal(6,5) #the upper quartile for the mean width values
    ninety_perc  : decimal(6,5) #the 90th percentile for the mean width values
    time_updated : timestamp    # the time at which the segmentation was performed
   """

In [ ]:
cas = ta3p100.ComponentAutoSegment()
seg_id = cas.fetch("segment_id")
print(len(seg_id))

In [ ]:
key_source = ta3p100.Compartment.Component & 'n_triangle_indices>100' & [dict(compartment_type=comp) for comp in ['Basal', 'Apical', 'Oblique', 'Dendrite']]
key_source

In [ ]:
ta3p100.Compartment.Component()

In [ ]:
(schema.jobs & "table_name='__component_auto_segment'").delete()